In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

In [ ]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    parameters = {
        # 'candidate': [2, 3, 4, 5],
        # 'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        'samples': [50, 100, 200, 400, 800, 1600, 3200, 6400],
        # 'dimensions': [50, 100, 200],
        # 'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
    }
    
    print('doc_id: %3s actual author: %4s author_id: %s' % (str(doc_id), str(row.author_id), str(authorList)))
    
    level = 'word'
    
    for idxp, paralist in parameters.iteritems():
        if idxp == 'samples':
            for idxl, val in enumerate(paralist):
                doc_id = row.doc_id
                
                author_id = row.author_id

                import ast
                authorList = ast.literal_eval(row.authorList)

                candidate = len(authorList)

                test = idxp # change before run

                level = "word"

                iterations = 30

                dropout = 0.4

                samples = val
                
                dimensions = 200

                loc = authorList.index(author_id)

                printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
                printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
                printstate += (("\niterations = %s, dropout = %s, ") % (str(iterations), str(dropout)))
                printstate += (("test = %s") % (str(test)))

                print("Current test: %s" % (str(printstate)))

                import UpdateDB as db

                case = db.checkOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions, samples = samples, 
                                      iterations = iterations, dropout = dropout, test = test)

                if case == False:

                    print("Running: %12s" % (str(printstate)))
        
                    import StyloNeural as Stylo
                    (labels_index, predYList, predY, history, samples) = Stylo.getResults(
                        doc_id = doc_id, authorList = authorList[:], 
                        level = level, glove = '../../glove/', dimensions = dimensions, 
                        samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 16 )
                
                    for key, auth in labels_index.iteritems():
                        if auth == author_id:
                            loc = key
                            
                    ans = predY[loc]

                    import UpdateDB as db
                    case = db.insertresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions, 
                                                 samples = samples, iterations = iterations, 
                                                 dropout = dropout, accuracy = ans, test = test)
                    
                    if case:
                        ans = 0.0
                        
                        if (predY.tolist().index(max(predY)) == loc):
                            ans = 1.0
                            
                        import UpdateDB as db
                        case = db.insertresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = dimensions, 
                                                     samples = samples, iterations = iterations, dropout = dropout, 
                                                     accuracy = ans, test = str(test + 'binary'))
                    
                    else:
                        print("Skipped: %12s test:   index: %2s" % (str(idxp), str(idxl)))
                        if samples != val:
                            break
                    
                    from IPython.display import clear_output

                    clear_output()
                else:
                    print("Skipped: %12s test:   index: %2s" % (str(idxp), str(idxl)))
                

Current test: doc_id = 168, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.4, test = samples
Execution completed
Running: doc_id = 168, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.4, test = samples
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 88
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (9928, 2)
Author:    27  Size:  2638
Author:    80  Size:  6423
Author:    81  Size:   867
Min: 867
Max: 6423
Authors [27, 80, 81].
Found 2601 texts.
Found 2601 labels.
Found 34696 unique tokens.
Shape of data tensor: (2601, 1000)
Shape of label tensor: (2601, 3)
Done compiling.
Train on 2340 samples, validate on 261 samples
Epoch 1/30
2340/2340 [==============================] - 12s - loss: 1.1006 - acc: 0.3303 - val_loss: 1.0971 - val_acc: 0.3716

In [ ]:
%tb